# AMAZON PC REVIEWS - NLP Topic Modeling
---

In [93]:
########################################
## Standard Python Modules/ Functions ##
########################################
import numpy as np
import pandas as pd
import re
import pprint
from collections import defaultdict
from collections import Counter
import pickle


import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

######################
## Plotting Modules ##
######################
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

%matplotlib inline

############################
## Classification Modules ##
############################
from imblearn.over_sampling import ADASYN
from smote_variants import DBSMOTE
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_curve,f1_score, fbeta_score, confusion_matrix, make_scorer

#########################################
## Natural Language Processing Modules ##
#########################################
import textstat
import nltk
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim import corpora, models, similarities, matutils
from gensim.models import Word2Vec

##########################################
## Clustering & Dimensionanlity Modules ##
##########################################
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.cluster import DBSCAN, KMeans, MeanShift

---

# Import Data:

In [4]:
# filepath = 'amazon_pc_reviews_cleaned (v2).csv'

In [5]:
# ## Import .csv file for PC Reviews
# pc_df = pd.read_csv(filepath, delimiter='\t')

In [6]:
# ## Drop all rows that contain any null values
# pc_df.dropna(how='any', inplace=True)

In [7]:
# ## Reset the index after dropping null values
# pc_df.reset_index(drop=True, inplace=True)

---

# TextStat Features:

#### Sentence Count (Review Body):

In [8]:
# pc_df['body_sentence_cnt'] = pd.Series(textstat.sentence_count(x) for x in pc_df['review_body'])

#### Flesch Reading Ease Score (Review Body):

The table below is an example of values.  
While the maximum score is 121.22, there is no limit on how low the score can be.  
A negative score is valid

In [9]:
%%html
<style>
table {float:left}
</style>

|Score	|Difficulty
|-------|-----------|
|90-100	|Very Easy
|80-89	|Easy
|70-79	|Fairly Easy
|60-69	|Standard
|50-59	|Fairly Difficult
|30-49	|Difficult
|0-29	|Very Confusing

In [10]:
# pc_df['body_flesch_reading_ease'] = pd.Series(textstat.flesch_reading_ease(x) for x in pc_df['review_body'])

#### The Flesch-Kincaid Grade Level (Review Body):

This is a grade formula in that a score of 9.3 means that a ninth grader would be able to read the document.

In [11]:
# pc_df['body_flesch_kincaid_grade'] = pd.Series(textstat.flesch_kincaid_grade(x) for x in pc_df['review_body'])

---

# Pickled DataFrame:
`To avoid expensive computation performed above when re-running Notebook.`

In [12]:
# pc_df.to_pickle('pc_reviews_data.pkl')

In [13]:
pc_df = pd.read_pickle('pc_reviews_data.pkl')

In [14]:
pc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6901412 entries, 0 to 6901411
Data columns (total 23 columns):
customer_id                  float64
review_id                    object
product_id                   object
product_parent               float64
product_title                object
product_category             object
star_rating                  float64
helpful_votes                float64
total_votes                  float64
vine                         int64
verified_purchase            int64
review_headline              object
review_body                  object
review_date                  object
clean_review_body            object
clean_review_headline        object
helpfullness_ratio           float64
wc_review_headline           float64
wc_review_body               float64
target                       int64
body_sentence_cnt            int64
body_flesch_reading_ease     float64
body_flesch_kincaid_grade    float64
dtypes: float64(10), int64(4), object(9)
memory usage

In [15]:
pc_df.head(2)

,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,...,review_date,clean_review_body,clean_review_headline,helpfullness_ratio,wc_review_headline,wc_review_body,target,body_sentence_cnt,body_flesch_reading_ease,body_flesch_kincaid_grade
0,51571469.0,R348IK2JAZJZ06,B004YLJNAE,805237341.0,Totoro on Branch Apple Macbook Vinyl Decal Lap...,PC,5.0,0.0,0.0,0,...,2013-06-07,this decal is as described i dont know about o...,perfect great service,0.0,3.0,20.0,0,2,69.79,6.0
1,29149498.0,R2RPUCGR6SR1R6,B0083PWAPW,270935683.0,"Kindle Fire HD 7"", Dolby Audio, Dual-Band Wi-Fi",PC,3.0,1.0,10.0,0,...,2013-06-04,i quickly went from five star elation to an al...,short lived honeymoon period,0.1,4.0,510.0,0,1,-439.75,201.8


---

# Text Pre-Processing:
    1. Establish Stop Words
    2. Define WordNet Lemmitizer with POS Tag

### World Lemmitizer Using "Parts of Speech" (POS) Tag

In [16]:
def get_pos_tag(word):
    """ 
    Determine POS tag for a word, and return it's first character.
    To be used in Lemmatize() function.
    """
    # Dictionary for "Parts of Speech" Tags
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    tag = pos_tag([word])[0][1][0].upper()
    
    return tag_dict.get(tag, wordnet.NOUN)
    

In [17]:
# ## Create class object for Lemmatization Tokenizer
# ## Uses WordNetLemmatizer

# class LemmaTokenizer(object):
#     def __init__(self):
#         self.wnl = WordNetLemmatizer()
#     def __call__(self, doc):
#         return [self.wnl.lemmatize(word, pos=get_pos_tag(word)) for word in word_tokenize(doc)]

In [18]:
## Create class object for Lemmatization Tokenizer
## Uses WordNetLemmatizer
## Filter output to only include Nouns

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(word, pos=get_pos_tag(word)) for word in word_tokenize(doc) 
                if re.compile(r'NN').search(pos_tag([word])[0][1])]

In [19]:
## Create class object for Stemmer Tokenizer
## Uses PorderStemmer
## Filter output to only include Nouns

class PStemmer(object):
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        return [self.ps.stem(word) for word in word_tokenize(doc) 
                if re.compile(r'NN').search(pos_tag([word])[0][1])] 

### Define All Stop Words

In [20]:
## Define "english" stopwords
stop_words = set(stopwords.words('english'))

In [21]:
## Add additional words to stopwords set
stop_words = stop_words.union({'pc', 'computer','laptop','macbook','notebook','ultrabook', 'mac', 'window', 'xps',
                               'hp','asus','lenovo','acer', 'dell','apple', 'air', 'pro', 'tablet'})

In [22]:
## New list of stop words that are Lemmatized (to match lemma-tokenized bag of words)
stop_lem = WordNetLemmatizer()
lem_stopwords = [stop_lem.lemmatize(word, pos=get_pos_tag(word)) for word in stop_words]

In [23]:
stop_stem = PorterStemmer()
stem_stopwords = [stop_stem.stem(word) for word in stop_words 
                if re.compile(r'NN').search(pos_tag([word])[0][1])]

---

# Amazon PC Sub-Category (Laptops):
In order to improve results of topic modeling, focus on finding topics relevant to reviews that are for laptops only. The "PC Category" from Amazon Reviews data contains many PC related accessory products.

In [24]:
laptops_df = pc_df.loc[(pc_df['product_title']
                  .str.contains(r'(Apple|HP|ASUS|Lenovo|Acer|Dell).*(-inch|-Inch).*(Laptop|laptop|Notebook|notebook|Ultrabook|ultrabook)'))]


/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.



In [25]:
# Separate majority and minority classes
df_majority = laptops_df[laptops_df['target']==0]
df_minority = laptops_df[laptops_df['target']==1]

In [26]:
df_majority.shape, df_minority.shape

((80548, 23), (5344, 23))

In [27]:
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=5344, # to match minority class
                                 random_state=42)  # reproducible results
    
# Combine minority class with downsampled majority class
laptop_downsampled_df = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
laptop_downsampled_df['target'].value_counts()

1    5344
0    5344
Name: target, dtype: int64

In [28]:
## Reset the index values after down sampling
laptop_downsampled_df.reset_index(drop=True, inplace=True)

---

# Word Vectorization:

In [29]:
helpful_laptop_reviews = laptop_downsampled_df.loc[(laptop_downsampled_df['target']==1) 
                                                  & (laptop_downsampled_df['helpfullness_ratio'] >0.8)
                                                  & (laptop_downsampled_df['total_votes'] >50)]['clean_review_body']

### Count Vectorizer (for LDA):

In [30]:
cv1 = CountVectorizer(stop_words=stop_words, ngram_range=(1,1), tokenizer=PStemmer())

doc_words = cv1.fit_transform(helpful_laptop_reviews).transpose()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['appl', 'asu', 'comput', 'need', 'ourselv', 'sha', 'xp', 'yourselv'] not in stop_words.



In [31]:
## Pickle cv1 CountVectorizer
pickle_out = open('laptops_cv1.pkl', 'wb') 

pickle.dump(cv1, pickle_out)                      
pickle_out.close() 

In [32]:
## Pickle doc_words Matrix
pickle_out = open('laptops_doc_words.pkl', 'wb') 

pickle.dump(doc_words, pickle_out)                      
pickle_out.close() 

In [33]:
## Open pickled cv1 CountVectorizer
pickle_in = open('laptops_cv1.pkl', 'rb') 

cv1 = pickle.load(pickle_in)

In [34]:
## Open pickled doc_words Matrix
pickle_in = open('laptops_doc_words.pkl', 'rb') 

doc_words = pickle.load(pickle_in)

### Convert to Gensim:
Convert our `doc_words` matrix to a `gensim`-friendly object called a Corpus.

In [35]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(doc_words)

In [36]:
## Creating mapping of matrix row id (document) to a word
## v = row id, k= vocab
id2word = dict((v, k) for k, v in cv1.vocabulary_.items())

---

# LDA Model:

### Create Model:

In [37]:
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaModel(corpus=corpus, num_topics=3, id2word=id2word, passes=40)

2019-11-21 00:37:21,481 : INFO : using symmetric alpha at 0.3333333333333333
2019-11-21 00:37:21,482 : INFO : using symmetric eta at 0.3333333333333333
2019-11-21 00:37:21,483 : INFO : using serial LDA version on this node
2019-11-21 00:37:21,486 : INFO : running online (multi-pass) LDA training, 3 topics, 40 passes over the supplied corpus of 851 documents, updating model once every 851 documents, evaluating perplexity every 851 documents, iterating 50x with a convergence threshold of 0.001000
2019-11-21 00:37:22,534 : INFO : -9.661 per-word bound, 809.7 perplexity estimate based on a held-out corpus of 851 documents with 189244 words
2019-11-21 00:37:22,534 : INFO : PROGRESS: pass 0, at document #851/851
2019-11-21 00:37:23,034 : INFO : topic #0 (0.333): 0.013*"screen" + 0.009*"batteri" + 0.009*"gb" + 0.008*"machin" + 0.008*"time" + 0.007*"keyboard" + 0.007*"drive" + 0.007*"use" + 0.007*"asu" + 0.006*"need"
2019-11-21 00:37:23,035 : INFO : topic #1 (0.333): 0.013*"drive" + 0.012*"use

2019-11-21 00:37:34,760 : INFO : topic #2 (0.333): 0.013*"comput" + 0.013*"screen" + 0.012*"use" + 0.010*"keyboard" + 0.008*"work" + 0.007*"im" + 0.007*"time" + 0.007*"batteri" + 0.007*"drive" + 0.007*"dont"
2019-11-21 00:37:34,760 : INFO : topic diff=0.038585, rho=0.316228
2019-11-21 00:37:35,754 : INFO : -7.224 per-word bound, 149.5 perplexity estimate based on a held-out corpus of 851 documents with 189244 words
2019-11-21 00:37:35,755 : INFO : PROGRESS: pass 9, at document #851/851
2019-11-21 00:37:36,198 : INFO : topic #0 (0.333): 0.012*"batteri" + 0.012*"gb" + 0.011*"screen" + 0.010*"machin" + 0.010*"appl" + 0.008*"use" + 0.008*"model" + 0.008*"asu" + 0.007*"drive" + 0.007*"life"
2019-11-21 00:37:36,199 : INFO : topic #1 (0.333): 0.017*"drive" + 0.013*"gb" + 0.010*"use" + 0.009*"screen" + 0.008*"batteri" + 0.007*"keyboard" + 0.007*"price" + 0.007*"usb" + 0.007*"ssd" + 0.006*"model"
2019-11-21 00:37:36,199 : INFO : topic #2 (0.333): 0.013*"screen" + 0.013*"comput" + 0.012*"use" + 

2019-11-21 00:37:47,526 : INFO : topic diff=0.016169, rho=0.229416
2019-11-21 00:37:48,494 : INFO : -7.215 per-word bound, 148.6 perplexity estimate based on a held-out corpus of 851 documents with 189244 words
2019-11-21 00:37:48,494 : INFO : PROGRESS: pass 18, at document #851/851
2019-11-21 00:37:48,918 : INFO : topic #0 (0.333): 0.013*"gb" + 0.013*"batteri" + 0.012*"appl" + 0.010*"machin" + 0.010*"screen" + 0.010*"model" + 0.008*"use" + 0.008*"drive" + 0.008*"comput" + 0.008*"life"
2019-11-21 00:37:48,918 : INFO : topic #1 (0.333): 0.018*"drive" + 0.014*"gb" + 0.009*"screen" + 0.009*"use" + 0.008*"batteri" + 0.008*"ssd" + 0.007*"usb" + 0.007*"keyboard" + 0.007*"price" + 0.007*"game"
2019-11-21 00:37:48,919 : INFO : topic #2 (0.333): 0.013*"screen" + 0.012*"comput" + 0.012*"use" + 0.011*"keyboard" + 0.008*"work" + 0.007*"im" + 0.007*"time" + 0.007*"batteri" + 0.007*"dont" + 0.007*"thing"
2019-11-21 00:37:48,919 : INFO : topic diff=0.015361, rho=0.223607
2019-11-21 00:37:49,892 : INF

2019-11-21 00:38:00,912 : INFO : -7.211 per-word bound, 148.2 perplexity estimate based on a held-out corpus of 851 documents with 189244 words
2019-11-21 00:38:00,913 : INFO : PROGRESS: pass 27, at document #851/851
2019-11-21 00:38:01,321 : INFO : topic #0 (0.333): 0.014*"gb" + 0.013*"appl" + 0.013*"batteri" + 0.011*"model" + 0.009*"machin" + 0.009*"screen" + 0.009*"drive" + 0.009*"comput" + 0.008*"use" + 0.008*"life"
2019-11-21 00:38:01,322 : INFO : topic #1 (0.333): 0.019*"drive" + 0.015*"gb" + 0.009*"screen" + 0.009*"use" + 0.008*"ssd" + 0.008*"batteri" + 0.008*"usb" + 0.007*"game" + 0.007*"price" + 0.007*"processor"
2019-11-21 00:38:01,323 : INFO : topic #2 (0.333): 0.013*"screen" + 0.012*"use" + 0.012*"comput" + 0.011*"keyboard" + 0.008*"work" + 0.008*"time" + 0.007*"im" + 0.007*"batteri" + 0.007*"dont" + 0.007*"thing"
2019-11-21 00:38:01,323 : INFO : topic diff=0.010719, rho=0.185695
2019-11-21 00:38:02,276 : INFO : -7.211 per-word bound, 148.2 perplexity estimate based on a he

2019-11-21 00:38:13,158 : INFO : PROGRESS: pass 36, at document #851/851
2019-11-21 00:38:13,556 : INFO : topic #0 (0.333): 0.014*"appl" + 0.014*"gb" + 0.013*"batteri" + 0.011*"model" + 0.009*"drive" + 0.009*"machin" + 0.009*"comput" + 0.009*"screen" + 0.008*"use" + 0.008*"life"
2019-11-21 00:38:13,557 : INFO : topic #1 (0.333): 0.020*"drive" + 0.016*"gb" + 0.009*"screen" + 0.009*"ssd" + 0.008*"use" + 0.008*"usb" + 0.008*"batteri" + 0.007*"game" + 0.007*"processor" + 0.007*"memori"
2019-11-21 00:38:13,557 : INFO : topic #2 (0.333): 0.013*"screen" + 0.012*"use" + 0.012*"comput" + 0.011*"keyboard" + 0.008*"work" + 0.008*"time" + 0.008*"im" + 0.007*"batteri" + 0.007*"dont" + 0.007*"thing"
2019-11-21 00:38:13,558 : INFO : topic diff=0.008354, rho=0.162221
2019-11-21 00:38:14,501 : INFO : -7.209 per-word bound, 147.9 perplexity estimate based on a held-out corpus of 851 documents with 189244 words
2019-11-21 00:38:14,501 : INFO : PROGRESS: pass 37, at document #851/851
2019-11-21 00:38:14,8

In [38]:
## Pickle LDA Model
pickle_out = open('laptop_lda_model.pkl', 'wb') 

pickle.dump(lda, pickle_out)                      
pickle_out.close() 

In [39]:
## Open pickled LDA Model
pickle_in = open('laptop_lda_model.pkl', 'rb') 

lda = pickle.load(pickle_in)

### Topic Space:

In [40]:
lda.print_topics()

2019-11-21 00:38:17,630 : INFO : topic #0 (0.333): 0.015*"appl" + 0.014*"gb" + 0.013*"batteri" + 0.011*"model" + 0.009*"drive" + 0.009*"comput" + 0.009*"machin" + 0.008*"screen" + 0.008*"use" + 0.008*"life"
2019-11-21 00:38:17,631 : INFO : topic #1 (0.333): 0.020*"drive" + 0.016*"gb" + 0.009*"screen" + 0.009*"ssd" + 0.008*"use" + 0.008*"usb" + 0.008*"batteri" + 0.007*"game" + 0.007*"processor" + 0.007*"memori"
2019-11-21 00:38:17,631 : INFO : topic #2 (0.333): 0.013*"screen" + 0.012*"use" + 0.012*"comput" + 0.011*"keyboard" + 0.008*"work" + 0.008*"time" + 0.008*"im" + 0.007*"batteri" + 0.007*"dont" + 0.007*"thing"


[(0,
  '0.015*"appl" + 0.014*"gb" + 0.013*"batteri" + 0.011*"model" + 0.009*"drive" + 0.009*"comput" + 0.009*"machin" + 0.008*"screen" + 0.008*"use" + 0.008*"life"'),
 (1,
  '0.020*"drive" + 0.016*"gb" + 0.009*"screen" + 0.009*"ssd" + 0.008*"use" + 0.008*"usb" + 0.008*"batteri" + 0.007*"game" + 0.007*"processor" + 0.007*"memori"'),
 (2,
  '0.013*"screen" + 0.012*"use" + 0.012*"comput" + 0.011*"keyboard" + 0.008*"work" + 0.008*"time" + 0.008*"im" + 0.007*"batteri" + 0.007*"dont" + 0.007*"thing"')]

---

# INDIVIDUAL REVIEW TOPICS:

## Word2Vec Model:

### Prepare Data for Word2Vec:

In [41]:
documents = helpful_laptop_reviews.tolist()

In [42]:
# texts = [[WordNetLemmatizer().lemmatize(word, pos=get_pos_tag(word)) 
#               for word in word_tokenize(doc) if word not in lem_stopwords] 
#           for doc in documents]

texts = [[PorterStemmer().stem(word) for word in word_tokenize(doc) 
                if re.compile(r'NN').search(pos_tag([word])[0][1]) 
                    if word not in stem_stopwords] for doc in documents]

In [43]:
# ## Pickle word2vec_texts
# pickle_out = open('laptop_word2vec_texts.pkl', 'wb') 

# pickle.dump(texts, pickle_out)                      
# pickle_out.close() 

In [44]:
# ## Open pickled doc_words Matrix
# pickle_in = open('word2vec_texts.pkl', 'rb') 

# texts = pickle.load(pickle_in)

### Create Model:
**`Word2Vec`** Model will contain vectors (in n-dimensional space, as defined by "size" argument in `Word2Vec()` function) for each word.

In [45]:
w2v_model = Word2Vec(texts, size=10, window=10, min_count=1, workers=2, sg=1)

2019-11-21 00:39:05,398 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-11-21 00:39:05,399 : INFO : collecting all words and their counts
2019-11-21 00:39:05,399 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-21 00:39:05,425 : INFO : collected 9534 word types from a corpus of 194293 raw words and 851 sentences
2019-11-21 00:39:05,425 : INFO : Loading a fresh vocabulary
2019-11-21 00:39:05,440 : INFO : effective_min_count=1 retains 9534 unique words (100% of original 9534, drops 0)
2019-11-21 00:39:05,440 : INFO : effective_min_count=1 leaves 194293 word corpus (100% of original 194293, drops 0)
2019-11-21 00:39:05,460 : INFO : deleting the raw counts dictionary of 9534 items
2019-11-21 00:39:05,460 : INFO : sample=0.001 downsamples 63 most-common words
2019-11-21 00:39:05,460 : INFO : downsampling leaves estimated 174914 word corpus (90.0% of prior 194293)
2019-11-21 00:39:05,473 : INFO : estimated required mem

In [46]:
list(w2v_model.wv.vocab.items())[:7]

[('im', <gensim.models.keyedvectors.Vocab at 0x7f3f94525780>),
 ('re-writ', <gensim.models.keyedvectors.Vocab at 0x7f3f942af7f0>),
 ('review', <gensim.models.keyedvectors.Vocab at 0x7f3f9426def0>),
 ('includ', <gensim.models.keyedvectors.Vocab at 0x7f3f9426dfd0>),
 ('tutori', <gensim.models.keyedvectors.Vocab at 0x7f403014b5f8>),
 ('fix', <gensim.models.keyedvectors.Vocab at 0x7f3f9426fef0>),
 ('warrante', <gensim.models.keyedvectors.Vocab at 0x7f3f9426fe48>)]

In [47]:
w2v_model.wv.most_similar('key' ,topn=8)

2019-11-21 00:39:11,717 : INFO : precomputing L2-norms of word weight vectors


[('press', 0.9897687435150146),
 ('backspac', 0.9845664501190186),
 ('sticki', 0.9724972248077393),
 ('button', 0.964767575263977),
 ('hit', 0.9637911319732666),
 ('spacebar', 0.963700532913208),
 ('miss', 0.960688054561615),
 ('pad', 0.9605816602706909)]

In [48]:
w2v_model.wv['netbook']

array([ 0.8218144 , -0.56024516, -0.36112645, -0.9073611 , -0.44240883,
        0.71504456, -0.6014483 ,  0.31318542,  0.723151  , -0.7115324 ],
      dtype=float32)

## Clustering Word Vectors:

### Create Sub-Sample of Word Vectors:

In [49]:
## Try to cluster word vectors of a similar product (ex. HP laptops)
sample_docs_idx = list(laptop_downsampled_df.loc[(laptop_downsampled_df['target']==1) 
                      & (laptop_downsampled_df['helpfullness_ratio'] >0.95)
                      & (laptop_downsampled_df['total_votes'] >50)].index)
len(sample_docs_idx)                                                                        

309

In [50]:
doc_vector = defaultdict(list)

for idx in range(0, len(texts)):
    for word in texts[idx]:
        doc_vector['word'].append(word)
        for i in range(0, 10):
            doc_vector['dim_'+str(i)].append(w2v_model.wv[word][i])
    
doc_vector_df = pd.DataFrame(doc_vector, columns=doc_vector.keys())

doc_vector_df.drop_duplicates(subset='word', keep="first", inplace=True)

In [51]:
doc_vector_df.reset_index(drop=True, inplace=True)

In [52]:
doc_vector_df

,word,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9
0,im,1.129741,-0.702177,0.279348,-0.316522,-0.297098,0.457756,-0.673519,0.637736,0.271848,-0.405192
1,re-writ,0.346847,0.059756,0.073903,-0.001137,-0.129828,0.079862,-0.122426,0.127874,0.196570,-0.150629
2,review,1.571441,-0.831710,0.099910,-0.251215,-0.228294,0.229136,-0.912391,0.451590,0.526504,0.172322
3,includ,1.214988,0.374126,-0.497575,-0.324801,-0.410497,0.867303,-0.862879,-0.148773,0.054680,-0.633312
4,tutori,1.068156,0.099612,0.471823,0.094565,-0.358540,0.671510,-0.553931,0.483963,0.203880,-0.272250
...,...,...,...,...,...,...,...,...,...,...,...
9529,crapshoot,0.341425,0.077017,0.238824,-0.039666,-0.212681,0.208834,-0.230976,0.156376,0.334858,-0.178740
9530,blood,0.437361,0.115140,0.195601,-0.000286,-0.186952,0.209366,-0.172158,0.131707,0.379465,-0.229281
9531,swapper,0.463549,0.025426,0.222530,-0.010263,-0.305340,0.161444,-0.179807,0.102001,0.373872,-0.265277
9532,featuresolut,0.530356,-0.012866,0.046315,-0.053651,-0.250189,0.139070,-0.255477,0.199893,0.290543,-0.343284


### Standard Scale for Clustering:

In [53]:
## Standard Scale values for clustering
scaler = StandardScaler()
scaler.fit(doc_vector_df.iloc[:, 1:])
X = scaler.transform(doc_vector_df.iloc[:, 1:])

In [54]:
X

array([[ 1.37204008, -2.67165913,  0.4331455 , ...,  1.46824513,
        -0.55909031,  0.08545109],
       [-1.24939597,  0.22590927, -0.3871196 , ..., -0.64374301,
        -0.89383422,  1.09606936],
       [ 2.85102318, -3.16426448, -0.28328231, ...,  0.69718012,
         0.57330545,  2.37819167],
       ...,
       [-0.85863157,  0.09535653,  0.20629382, ..., -0.75091371,
        -0.10541161,  0.64091676],
       [-0.63493466, -0.05026457, -0.4972653 , ..., -0.34541762,
        -0.47595747,  0.33122546],
       [-0.87607852,  0.23183971, -0.2097092 , ..., -0.13894531,
        -0.6735493 , -0.02033236]])

## DBSCAN Clustering:

In [55]:
dbclusters = DBSCAN(eps=2, min_samples=4).fit(X)

In [56]:
Counter(dbclusters.labels_)

Counter({0: 8293, 1: 4, -1: 1221, 2: 5, 3: 3, 6: 2, 4: 3, 5: 3})

In [ ]:
cluster_pred = list(dbclusters.labels_)
cluster3_idx = []
for idx, item in enumerate(dbclusters.labels_):
    if item == 3:
        cluster3_idx.append(idx)
        #cluster_pred.pop(item)

In [ ]:
for idx in cluster3_idx:
    print(doc_vector_df.loc[idx]['word'])

---

## Latent Semantic Analysis (LSA) w/ Count Vectorizer:

In [81]:
cv2 = CountVectorizer(min_df=3, max_df=len(helpful_laptop_reviews),
                                           stop_words=stem_stopwords, ngram_range=(1,1), tokenizer=PStemmer())
cv2

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=851, max_features=None, min_df=3,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=["mustn't", 'doesn', 'i', "couldn't", "shan't",
                            'window', 'd', 'didn', 'll', 't', 're', "don't",
                            'weren', 'your', "should'v", 'ma', 'aren',
                            "wouldn't", "you'v", 'their', 'won', 'haven',
                            "isn't", 've', "hasn't", "haven't", 'comput',
                            'tablet', 'pro', 'notebook', ...],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<__main__.PStemmer object at 0x7f3f147de6d8>,
                vocabulary=None)

In [82]:
doc_words2 = cv2.fit_transform(helpful_laptop_reviews)
doc_words2

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['need', 'r', 'sha', 'v'] not in stop_words.



<851x3326 sparse matrix of type '<class 'numpy.int64'>'
	with 99303 stored elements in Compressed Sparse Row format>

In [83]:
lsa = TruncatedSVD(3)
doc_topic = lsa.fit_transform(doc_words2)
lsa.explained_variance_ratio_

array([0.18061127, 0.04756141, 0.04182004])

In [84]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [85]:
display_topics(lsa, cv2.get_feature_names(), 8)


Topic  0
use, screen, drive, batteri, gb, keyboard, time, work

Topic  1
x, control, mhz, set, cpu, batteri, ddr, hour

Topic  2
gb, x, drive, cpu, mhz, model, batteri, core


In [86]:
Vt = pd.DataFrame(doc_topic.round(5),
             index = helpful_laptop_reviews,
             columns = ["c1","c2","c3"])
Vt

c1       c2       c3
clean_review_body                                                             
im re-writing this review to include a tutorial...   8.14606  1.56123 -1.20793
i got this after a real odyssey searching for a...  10.93385  2.50282 -1.81780
on paper this laptop has a lot going for it a r...  11.18435  0.83345 -1.15721
just bought a unit from a local store i agree w...  13.39921  4.24578 -6.14111
acer isnt an unfamiliar brand around our house ...  10.23887  2.32986 -3.14220
...                                                      ...      ...      ...
asus began the netbook era with a very small co...  13.30356  0.93076 -1.88208
st solid construction little to no keyboard fle...  13.78395  5.54132 -2.86404
amazing notebook for under fast shipping from a...   8.93409  1.37534 -0.72662
i was looking for a lightweight lb or less touc...  14.38589  2.55690 -6.33762
well its expensive more expensive than any tabl...  14.06633  2.51567  1.23604

[851 rows x 3 columns]

---

# Latent Semantic Analysis (LSA) w/ TF-IDF:

In [87]:
tfidf = TfidfVectorizer(min_df=3, max_df=len(helpful_laptop_reviews),
                        stop_words=stem_stopwords, ngram_range=(1,1), tokenizer=PStemmer())
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=851, max_features=None,
                min_df=3, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=["mustn't", 'doesn', 'i', "couldn't", "shan't",
                            'window', 'd', 'didn', 'll', 't', 're', "don't",
                            'weren', 'your', "should'v", 'ma', 'aren',
                            "wouldn't", "you'v", 'their', 'won', 'haven',
                            "isn't", 've', "hasn't", "haven't", 'comput',
                            'tablet', 'pro', 'notebook', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<__main__.PStemmer object at 0x7f3f143c43c8>,
                use_idf=True, vocabulary=None)

In [88]:
doc_words3 = tfidf.fit_transform(helpful_laptop_reviews)
doc_words3

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['need', 'r', 'sha', 'v'] not in stop_words.



<851x3326 sparse matrix of type '<class 'numpy.float64'>'
	with 99303 stored elements in Compressed Sparse Row format>

In [89]:
lsa2 = TruncatedSVD(3)
doc_topic = lsa2.fit_transform(doc_words3)
lsa2.explained_variance_ratio_

array([0.01098931, 0.01343031, 0.0094722 ])

In [90]:
display_topics(lsa2, tfidf.get_feature_names(), 10)


Topic  0
screen, drive, use, gb, batteri, keyboard, machin, im, work, time

Topic  1
gb, drive, ssd, model, ram, memori, core, processor, intel, usb

Topic  2
machin, servic, custom, issu, problem, drive, amazon, support, repair, driver


In [91]:
Vt = pd.DataFrame(doc_topic.round(5),
             index = helpful_laptop_reviews,
             columns = ["c1","c2","c3"])
Vt

c1       c2       c3
clean_review_body                                                            
im re-writing this review to include a tutorial...  0.25544 -0.08779  0.05999
i got this after a real odyssey searching for a...  0.26146 -0.01964 -0.15902
on paper this laptop has a lot going for it a r...  0.37315  0.02112 -0.16773
just bought a unit from a local store i agree w...  0.33648 -0.17434 -0.16403
acer isnt an unfamiliar brand around our house ...  0.29625 -0.03582 -0.05707
...                                                     ...      ...      ...
asus began the netbook era with a very small co...  0.32755  0.00624 -0.04542
st solid construction little to no keyboard fle...  0.28632 -0.05072 -0.07515
amazing notebook for under fast shipping from a...  0.23115 -0.03983  0.04384
i was looking for a lightweight lb or less touc...  0.38504 -0.14942 -0.07069
well its expensive more expensive than any tabl...  0.26148 -0.00019 -0.08608

[851 rows x 3 columns]

---

# Non-Matrix Factorization (NMF) w/ Count Vectorizer:

In [94]:
nmf_model = NMF(3)
doc_topics = nmf_model.fit_transform(doc_words2)

In [95]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["c1","c2","c3"],
             columns = cv2.get_feature_names())
topic_word

,a-m,aa,aback,abg,abgn,abil,abit,absolut,abstract,absurd,...,yr,z,zenbook,zero,zillion,zip,zippi,zombi,zone,zoom
c1,0.002,0.046,0.005,0.001,0.022,0.176,0.000,0.026,0.00,0.019,...,0.000,0.251,0.401,0.159,0.006,0.006,0.000,0.006,0.016,0.165
c2,0.004,0.029,0.001,0.000,0.001,0.020,0.127,0.004,0.00,0.000,...,0.000,0.067,0.023,0.000,0.000,0.003,0.000,0.000,0.000,0.001
c3,0.014,0.000,0.004,0.021,0.018,0.024,0.000,0.000,0.03,0.000,...,0.036,0.000,0.043,0.000,0.001,0.008,0.108,0.000,0.008,0.000


In [97]:
display_topics(nmf_model, cv2.get_feature_names(), 10)


Topic  0
screen, use, keyboard, work, dont, time, machin, thing, im, review

Topic  1
x, batteri, cpu, mhz, control, ddr, set, hour, graphic, core

Topic  2
drive, gb, model, ssd, inch, port, use, usb, ram, need


In [ ]:
nmf_model.